# Simple XML

This notebook goes over how run an agent that uses XML to format steps

In [5]:
# !pip install openai agents_deconstructed

In [1]:
from langchain.agents import tool
from langchain.schema.agent import AgentFinish
from langchain.callbacks.manager import (
    trace_as_chain_group,
)
from langchain import hub
from langchain.chat_models import ChatAnthropic, ChatOpenAI

In [2]:
from agents_deconstructed import xml
from agents_deconstructed.format_tools import format_tools_description

In [4]:
# Create our 'agent' - this is a runnable
prompt = hub.pull("shoggoth13/xml-agent")
agent = prompt | ChatOpenAI(temperature=0).bind(stop=["</tool_input>", "</final_answer>"]) | xml.XMLAgentOutputParser()

In [5]:
# Create a dummy tool
@tool
def run_code(query: str):
    """Input should be Python code. Use this to do math."""
    return eval(query)

In [6]:
# Use this to group all traces together
with trace_as_chain_group("agent-run") as group_manager:
    # Keep track of steps taken
    steps = []
    # Get the first response from the agent
    action = agent.invoke({
        "question": "whats 2 + 2",
        "tools": format_tools_description([run_code]),
        "intermediate_steps": xml.format_steps(steps)
    }, config={"callbacks": group_manager})

    # While it's not finished, iterate
    while not isinstance(action, AgentFinish):
        # Select the tool to use
        tool = {
            "run_code": run_code,
        }[action.tool]
        # Run the tool
        observation = tool.run(action.tool_input, callbacks=group_manager)
        # Construct intermediate steps
        steps += [(action, observation)]
        # Log the steps
        print(steps)
        # Do the next iteration
        action = agent.invoke({
            "question": "whats 2 + 2",
            "tools": format_tools_description([run_code]),
            "intermediate_steps": xml.format_steps(steps)
        }, config={"callbacks": group_manager})

[(AgentAction(tool='run_code', tool_input='2 + 2', log='<tool>run_code</tool><tool_input>2 + 2'), 4)]


In [7]:
print(action)

AgentFinish(return_values={'output': '2 + 2 equals 4'}, log='<final_answer>2 + 2 equals 4')
